In [40]:
import numpy as np

training_data_length = 1000
training_data = np.zeros((training_data_length, 4), dtype=int)

searched_patterns = [[1, 0, 0, 1]]

for i in range(training_data_length):
    pattern = np.array([1, 1, 0, 0])
    np.random.shuffle(pattern)
    training_data[i] = pattern

def getCorrectResults(data):
    return np.array([1 if list(pattern) in searched_patterns else 0 for pattern in data])

correct_results_training_data = getCorrectResults(training_data)

In [41]:
import tensorflow.keras as keras


NN = keras.Sequential()
NN.add(keras.Input(shape=(4,)))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=100)

NN.summary()

weights = NN.get_weights()

# print the weights
for i, layer_weights in enumerate(weights):
    print(f"Layer {i}:")
    for j, weight in enumerate(layer_weights):
        print(f"Weight {j}: {weight}")

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6544 - loss: 0.6084   
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step - accuracy: 0.6681 - loss: 0.5795
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - accuracy: 0.6708 - loss: 0.5500
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.6666 - loss: 0.5368
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - accuracy: 0.5840 - loss: 0.5147
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - accuracy: 0.6536 - loss: 0.5050
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.6952 - loss: 0.4765
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.8361 - loss: 0.4644
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.8532 - loss: 0.4443
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.8241 - loss: 0.4418
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - accuracy: 0.8422 - loss: 0.4226
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17 (72.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12 (52.00 B)

Layer 0:
Weight 0: [1.1960047]
Weight 1: [-2.3823605]
Weight 2: [-2.232731]
Weight 3: [1.076004]
Layer 1:
Weight 0: -1.2346510887145996


Now we want to test the Network with a  new list of patterns.

In [42]:
test_data = np.copy(training_data)
np.random.shuffle(test_data)

# NN.predict returns an array with shape=(trainging_data_length = 1000, output_neurons = 1) 
# so we flatten it to get a 1D array with our predictions
results_on_test_data = NN.predict(test_data).flatten() 
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print("all test data correct, when rounded: ", all_correct)
print(f"maximum difference from correct result: {max_difference}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
all test data correct, when rounded:  True
maximum difference from correct result: 0.261660099029541
